## 9.1 GAN 기본 모델 구현하기

GAN(Generative Adversarial Network) : autoencoder와 같이 결과물을 생성하는 생성 모델

- adversarial한 두 신경망을 경쟁시켜가며 결과물 생성 방법을 학습

1. 실제 이미지를 구조 Discriminator에게 이 이미지가 진짜 임을 판단하게 함(학습시킨다고 생각)

2. Generator를 통해 noise(random vector)로 부터 임의의 이미지를 만듦

3. 만들어진 이미지를 다시 Discriminator를 통해 진짜 이미지인지 판단하게 함

-> **Generator는 Discriminator를 속여 진짜처럼 보이게 하고, Discriminator는 generator가 만든 이미지를 최대한 가짜라고 구분하도록 훈련!**

=> 경쟁을 통해 Generator는 실제 이미지와 상당히 비슷한 이미지 생성 가능!

In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

my_path = os.path.abspath('C:\\Users\\soual\\3Mdeeplearning\\9장 딥러닝의 미래 GAN\\samples')

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [26]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

hyperparameter 설정

n_noise : generator의 input으로 사용할 vector size! (latent vector)

-> random noise를 입력하여 이 noise로 부터 손글씨 이미지를 무작위로 생성하도록!

In [27]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

GAN 역시 unsupervised learning! -> Y 사용 안함

Discriminator에 넣을 이미지 : 실제 이미지 & Generator가 생성한 가짜 이미지

가자 이미지 : noise 에서 생성 => noise를 입력할 placeholder Z 추가!

In [28]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

#### Generator 신경망에 사용할 변수 설정

W1, b1 : hidden layer로 출력하기 위한 변수

W2, b2 : output layer에 사용할 변수 -> W2에 사용할 변수 크기 = 28X28 = 784

In [29]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

#### Discriminator 신경망에 사용할 변수 설정

hidden layer는 generator와 동일하게 구성

Discriminator : 진짜와 얼마나 가까운가를 판단하는 값으로 0~1 사이 값 출력 => scalar 값 1개 출력

In [30]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

#### Generator NN 

무작위로 생성한 noise를 받아 weight와 bias를 반영하여 hidden layer를 만듦

hidden layer에서 실제 image와 같은 크기의 결과값을 출력!

In [31]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

#### Discriminator NN

같은 구성! but 0~1사이 scalar 값 하나 출력하기 위해 활성화 함수로 sigmoid 함수 사용

In [32]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size = (batch_size, n_noise))

random noise를 만들어주는 유틸리티 함수 설정

In [33]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

노이즈 Z를 이용해 가짜 이미지를 만들 generator G를 만듦

이 G가 만든 가짜 이미지와 진짜 이미지 X를 각각 Discriminator에 넣어 입력한 이미지가 진짜인지 판별!

-> 잘 학습시킬 경우 생성자가 실제에 가까운 이미지를 만들 수 있음

In [34]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))

#### Cost function 설정

2개 필요! 

1. Generator가 만든 이미지를 Discriminator가 가짜라고 판단하도록 하는 cost (경찰이 학습해야!)

-> D_real은 1에 가까워야 하며(진짜라고 판별), D_gene은 0에 가까워야 함(가짜라고 판별)

-> D_real과 1-D_gene 을 더한 값을 손실값으로! -> 이를 최대화하면 경찰(Discriminator) 학습

In [35]:
loss_G = tf.reduce_mean(tf.log(D_gene))

2. Generator가 만든 이미지를 Discriminator가 진짜라고 판단하도록 하는 cost (위조지폐범이 학습해야!)

-> 가짜 이미지 판별값 D_gene을 1에 가깝게 만들어야함! => 가짜 이미지를 넣어도 진짜 같다고 판별해야함

-> D_gene을 그대로 넣어 이를 손실값으로 사용

-> 이 값을 최대화! -> 이를 최대화하면 위조지폐범(Generator) 학습

**GAN의 학습 = loss_G 와 loss_D 모두 최대화**

loss_D와 loss_G : 서로 연관 -> 두 loss가 항상 같이 증가하는 경향을 보이지는 않음! 

loss_D가 증가 -> loss_G 감소 / loss_G 증가 -> loss_D 감소 => 경쟁관계

In [36]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

#### loss 값을 이용해 학습!

- loss_D : Discriminator NN에 사용되는 변수들만 사용 -> 최적화

- loss_G : Generator NN에 사용되는 변들만 사용 -> 최적화

=> loss_D 학습시 Generator 변하지 않음 / loss_G 학습시 Discriminator 변하지 않음

In [37]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

#### 최적화 함수 구성

loss 최대화 필요! -> optimizer에 사용가능한 함수 : minimize 뿐! -> 최적화하려는 loss_D와 loss_G에 음수 부호 붙여줌!

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0,0

#### 신경망 학습

session 설정과 minibatch를 위한 코드 설정, loss_D와 loss_G의 결과값을 받을 변수 설정 -> minibatch로 학습 반복

In [41]:
for epoch in range(total_epoch):    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Z: noise})
        
    print('Epoch: ', '%04d' % epoch,
         'D_loss: {:.4}'.format(loss_val_D), 
         'G_loss: {:.4}'.format(loss_val_G))
    
    
    #확인용 이미지 생성
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize = (sample_size,1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches = 'tight')
        plt.close(fig)
        
print('최적화 완료!')

Epoch:  0000 D_loss: -0.6995 G_loss: -1.996
Epoch:  0001 D_loss: -0.7599 G_loss: -2.145
Epoch:  0002 D_loss: -0.7631 G_loss: -1.936
Epoch:  0003 D_loss: -0.6436 G_loss: -1.813
Epoch:  0004 D_loss: -0.7147 G_loss: -2.153
Epoch:  0005 D_loss: -0.7552 G_loss: -2.136
Epoch:  0006 D_loss: -0.6448 G_loss: -2.016
Epoch:  0007 D_loss: -0.6783 G_loss: -1.833
Epoch:  0008 D_loss: -0.9959 G_loss: -1.854
Epoch:  0009 D_loss: -0.7129 G_loss: -1.797
Epoch:  0010 D_loss: -0.7516 G_loss: -2.054
Epoch:  0011 D_loss: -0.6189 G_loss: -2.0
Epoch:  0012 D_loss: -0.6259 G_loss: -1.927
Epoch:  0013 D_loss: -0.6345 G_loss: -1.967
Epoch:  0014 D_loss: -0.7756 G_loss: -2.078
Epoch:  0015 D_loss: -0.6463 G_loss: -1.888
Epoch:  0016 D_loss: -0.7917 G_loss: -1.887
Epoch:  0017 D_loss: -0.6679 G_loss: -2.052
Epoch:  0018 D_loss: -0.7249 G_loss: -2.129
Epoch:  0019 D_loss: -0.6273 G_loss: -2.207
Epoch:  0020 D_loss: -0.6147 G_loss: -2.277
Epoch:  0021 D_loss: -0.7109 G_loss: -1.973
Epoch:  0022 D_loss: -0.5671 G_los

Discriminator : X값, Generator : Z값(noise) 받음 

-> noise 생성해주는 get_noise 함수 사용 => batch_size 만큼 만들고 입력! -> 학습!

##### 확인용 이미지 생성

학습 결과 확인하는 코드 작성!

학습이 잘 되는지 0번째, 9번째, 19번째,... 이미지 생성하연 눈으로 직접 확인! -> 결과 확인 코드 loop 안에 작성! 

1. noize를 만들어 생성자 G에 넣어 결과값 만듦

2. 결과값을 28X28 크기의 가짜 이미지로 만들어 sample 폴더에 저장!(sample 폴더는 미리 만들어져 있어야함)
